In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
task_names = [
#     "COLA", 
#     "SST2",
#     "MNLI",
    "RTE",
#     "WNLI",
#     "QQP",
#     "MRPC",
#     "STSB",
#     "QNLI",
#     "SPACY_POS",
#     "SPACY_NER",
#     "THIRD",
#     "BLEU",
]


In [3]:
SEED = 1

## Load previously trained model
Hint: make sure the `bert_model` is initialized correctly!

In [4]:
task_kwargs = {
    "dl_kwargs": {"batch_size": 8},
#     freeze_bert=True,
    "bert_model": 'bert-large-cased',
    "max_len": 200   
}

In [5]:
%%time
from metal.mmtl.glue.glue_tasks import create_glue_tasks_payloads

# Create tasks and payloads
tasks, payloads = create_glue_tasks_payloads(task_names, **task_kwargs)

Using random seed: 654000
Loading RTE Dataset



CPU times: user 29.5 s, sys: 7.01 s, total: 36.6 s
Wall time: 32.6 s


In [6]:
tasks, payloads

([ClassificationTask(name=RTE, loss_multiplier=1.0)],
 [Payload(RTE_train: tasks=[RTE], split=train),
  Payload(RTE_valid: tasks=[RTE], split=valid),
  Payload(RTE_test: tasks=[RTE], split=test)])

In [7]:
from metal.mmtl.metal_model import MetalModel

model = MetalModel(tasks, seed=SEED, verbose=False)

In [8]:
import os
import torch
model_dir = '/dfs/scratch0/chami/metal/metal/mmtl/aws/output/2019_03_14_01_58_14/0/logdir/bert_large/QNLI.STSB.MRPC.QQP.RTE.MNLI.SST2.COLA.WNLI_09_15_09'
model_path = os.path.join(model_dir, 'best_model.pth')
device = torch.device(f"cuda:0")
model.load_state_dict(torch.load(model_path, map_location=device)["model"], strict=False)

#### Sanity check that task head is trained!

In [9]:
model.score(payloads[1])

{'RTE/RTE_valid/accuracy': 0.8231046931407943}

## Define slices for evaluation

In [10]:
%%time
from metal.mmtl.glue.glue_tasks import create_glue_tasks_payloads

# define slices
slice_dict = {  # A map of the slices that apply to each task
#    "COLA": ["ends_with_question_mark"]
    "RTE": ["dash_semicolon", "more_people"]
}

# Create tasks and payloads
task_kwargs.update({"slice_dict": slice_dict})
_, payloads_slice = create_glue_tasks_payloads(
    task_names, **task_kwargs
)

Using random seed: 128037
Loading RTE Dataset



Added label_set with 2490 labels for task RTE:dash_semicolon to payload RTE_train.
Added label_set with 2490 labels for task RTE:more_people to payload RTE_train.
Added label_set with 277 labels for task RTE:dash_semicolon to payload RTE_valid.
Added label_set with 277 labels for task RTE:more_people to payload RTE_valid.
Added label_set with 3000 labels for task RTE:dash_semicolon to payload RTE_test.
Added label_set with 3000 labels for task RTE:more_people to payload RTE_test.
CPU times: user 24.4 s, sys: 14.5 s, total: 38.8 s
Wall time: 39.8 s


In [11]:
payloads_slice

[Payload(RTE_train: tasks=[RTE,RTE:dash_semicolon,RTE:more_people], split=train),
 Payload(RTE_valid: tasks=[RTE,RTE:dash_semicolon,RTE:more_people], split=valid),
 Payload(RTE_test: tasks=[RTE,RTE:dash_semicolon,RTE:more_people], split=test)]

#### Sanity check the number of examples in the train set

In [12]:
import numpy as np
def count_num_labels(labels):
    return np.sum(np.array(labels) != 0)

In [13]:
dataset = payloads_slice[0].data_loader.dataset
for labelset_name, labels in dataset.labels.items():
    print(labelset_name, count_num_labels(labels))

RTE 2490
RTE:dash_semicolon 1003
RTE:more_people 64


### Evaluate baseline model on the slice of interest

In [14]:
model.score(payloads_slice[1])

{'RTE/RTE_valid/accuracy': 0.8231046931407943,
 'RTE:dash_semicolon/RTE_valid/accuracy': 0.7413793103448276,
 'RTE:more_people/RTE_valid/accuracy': 0.6666666666666666}

## Finetune model on slice of interest

In [15]:
from metal.mmtl.metal_model import MetalModel

from metal.mmtl.trainer import MultitaskTrainer
trainer = MultitaskTrainer(seed=SEED)

Uncomment below to only finetune on the slices, not the original task


In [16]:
# # only finetune on the slices, not the original task
# for p in payloads_slice:
#     p.task_names.remove('COLA')
# payloads_slice

NOTE: We are training on a different set of payloads than we initialized the model with.

In [17]:
trainer.train_model(
    model,
    payloads_slice,
    checkpoint_metric="RTE/RTE_valid/accuracy",
    checkpoint_metric_mode="max",
    checkoint_best=True,
    writer="tensorboard",
    optimizer="adamax",
    lr=5e-5,
    l2=1e-3,
    log_every=0.1, 
    score_every=0.1,
    n_epochs=3,
    progress_bar=True,
    checkpoint_tasks=True,
    checkpoint_cleanup=False
)

Beginning train loop.
Expecting a total of approximately 2496 examples and 312 batches per epoch from 1 payload(s) in the train split.
Writing config to /dfs/scratch0/vschen/metal-mmtl/logs/2019_03_19/15_26_35/config.json


/dfs/scratch0/vschen/metal-mmtl/metal/mmtl/trainer.py:578: UserWarning: checkpoint_tasks setting does not have the same thorough error checking that the normal checkpoint operation has, so you may accidentally be trying to checkpoint metrics that aren't going to be found in the metrics_dict if you're not careful.
  warnings.warn(msg)


/dfs/scratch0/vschen/metal-mmtl/metal/mmtl/trainer.py:489: UserWarning: You requested glue score but have fewer than 9 tasks. Be aware.
  warnings.warn(msg)


[0.10 epo]: RTE:[train/loss=2.76e-01, RTE_valid/accuracy=8.30e-01] RTE:dash_semicolon:[train/loss=3.01e-01, RTE_valid/accuracy=7.67e-01] RTE:more_people:[train/loss=3.71e-01, RTE_valid/accuracy=6.67e-01] model:[train/loss=2.85e-01, train/lr=5.00e-05, valid/glue=8.30e-01]
Saving model at iteration 0.10 with best (max) score RTE/RTE_valid/accuracy=0.830
Saving model at iteration 0.10 with best (max) score RTE:dash_semicolon/RTE_valid/accuracy=0.767
Saving model at iteration 0.10 with best (max) score RTE:more_people/RTE_valid/accuracy=0.667
[0.21 epo]: RTE:[train/loss=2.84e-01, RTE_valid/accuracy=8.01e-01] RTE:dash_semicolon:[train/loss=2.73e-01, RTE_valid/accuracy=7.24e-01] RTE:more_people:[train/loss=4.61e-01, RTE_valid/accuracy=5.83e-01] model:[train/loss=2.87e-01, train/lr=5.00e-05, valid/glue=8.01e-01]
[0.31 epo]: RTE:[train/loss=3.86e-01, RTE_valid/accuracy=8.23e-01] RTE:dash_semicolon:[train/loss=3.70e-01, RTE_valid/accuracy=7.41e-01] RTE:more_people:[train/loss=1.38e+00, RTE_vali

[1.03 epo]: RTE:[train/loss=2.46e-01, RTE_valid/accuracy=8.12e-01] RTE:dash_semicolon:[train/loss=2.60e-01, RTE_valid/accuracy=7.33e-01] RTE:more_people:[train/loss=1.85e-02, RTE_valid/accuracy=6.67e-01] model:[train/loss=2.45e-01, train/lr=5.00e-05, valid/glue=8.12e-01]
[1.13 epo]: RTE:[train/loss=1.98e-01, RTE_valid/accuracy=8.23e-01] RTE:dash_semicolon:[train/loss=3.39e-02, RTE_valid/accuracy=7.50e-01] RTE:more_people:[train/loss=5.51e-03, RTE_valid/accuracy=6.67e-01] model:[train/loss=1.49e-01, train/lr=5.00e-05, valid/glue=8.23e-01]
[1.23 epo]: RTE:[train/loss=1.57e-01, RTE_valid/accuracy=8.05e-01] RTE:dash_semicolon:[train/loss=1.76e-01, RTE_valid/accuracy=7.41e-01] RTE:more_people:[train/loss=1.41e-03, RTE_valid/accuracy=6.67e-01] model:[train/loss=1.59e-01, train/lr=5.00e-05, valid/glue=8.05e-01]
[1.33 epo]: RTE:[train/loss=2.68e-01, RTE_valid/accuracy=7.94e-01] RTE:dash_semicolon:[train/loss=2.82e-01, RTE_valid/accuracy=7.33e-01] RTE:more_people:[train/loss=4.14e-01, RTE_valid

[2.05 epo]: RTE:[train/loss=3.70e-01, RTE_valid/accuracy=8.12e-01] RTE:dash_semicolon:[train/loss=2.61e-01, RTE_valid/accuracy=7.50e-01] RTE:more_people:[train/loss=2.64e-02, RTE_valid/accuracy=5.83e-01] model:[train/loss=3.39e-01, train/lr=5.00e-05, valid/glue=8.12e-01]
[2.15 epo]: RTE:[train/loss=2.28e-01, RTE_valid/accuracy=8.30e-01] RTE:dash_semicolon:[train/loss=1.46e-01, RTE_valid/accuracy=7.76e-01] RTE:more_people:[train/loss=6.91e-02, RTE_valid/accuracy=5.83e-01] model:[train/loss=2.01e-01, train/lr=5.00e-05, valid/glue=8.30e-01]
[2.26 epo]: RTE:[train/loss=1.64e-01, RTE_valid/accuracy=7.91e-01] RTE:dash_semicolon:[train/loss=1.28e-01, RTE_valid/accuracy=7.24e-01] RTE:more_people:[train/loss=4.07e-03, RTE_valid/accuracy=5.83e-01] model:[train/loss=1.52e-01, train/lr=5.00e-05, valid/glue=7.91e-01]
[2.36 epo]: RTE:[train/loss=4.50e-01, RTE_valid/accuracy=8.09e-01] RTE:dash_semicolon:[train/loss=2.67e-01, RTE_valid/accuracy=7.59e-01] RTE:more_people:[train/loss=8.65e-04, RTE_valid

#### Did we improve?

TODO: The final metrics score printed above may be inaccurate... 

Better way to evaluate is by re-evaluating on the checkpoint with the best weights.

In [18]:
from metal.mmtl.metal_model import MetalModel
model.score(payloads_slice[1])

{'RTE/RTE_valid/accuracy': 0.8411552346570397,
 'RTE:dash_semicolon/RTE_valid/accuracy': 0.7931034482758621,
 'RTE:more_people/RTE_valid/accuracy': 0.75}